In [1]:
# I/O
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import geopandas as gpd
import requests
from io import BytesIO
from zipfile import ZipFile
from datetime import datetime, timedelta

In [2]:
# URL of the zipped data file
urls = ['https://s3.amazonaws.com/capitalbikeshare-data/202401-capitalbikeshare-tripdata.zip',
        'https://s3.amazonaws.com/capitalbikeshare-data/202402-capitalbikeshare-tripdata.zip', 
       'https://s3.amazonaws.com/capitalbikeshare-data/202403-capitalbikeshare-tripdata.zip']

In [3]:
# List to store DataFrames
dataframes = []
# Loop through each URL
for url in urls:
    # Download the zipped file
    response = requests.get(url)
    zipfile = ZipFile(BytesIO(response.content))
    
    # List files in the zip file
    file_name = zipfile.namelist()[0]
    
    # Read the desired file into a pandas DataFrame
    with zipfile.open(file_name) as file:
        combined_df = pd.read_csv(file)
        dataframes.append(combined_df)

# Concatenate all DataFrames into one
combined_combined_df = pd.concat(dataframes, ignore_index=True)

In [4]:
# Display the DataFrame
combined_combined_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,748A93D7DE8A41CD,classic_bike,2024-01-25 15:49:59,2024-01-25 15:52:35,1st & O St NW,31519.0,1st & L St NW,31677.0,38.908643,-77.012365,38.903819,-77.011987,member
1,75CBFD136F06305B,classic_bike,2024-01-02 16:44:58,2024-01-02 16:53:25,1st & O St NW,31519.0,4th & College St NW,31138.0,38.908643,-77.012365,38.921233,-77.018135,member
2,0536C9720F87E04C,classic_bike,2024-01-24 15:40:15,2024-01-24 15:43:55,1st & O St NW,31519.0,1st & L St NW,31677.0,38.908643,-77.012365,38.903819,-77.011987,member
3,9E17390C218783B5,classic_bike,2024-01-04 15:35:00,2024-01-04 15:37:35,1st & O St NW,31519.0,1st & L St NW,31677.0,38.908643,-77.012365,38.903819,-77.011987,member
4,00727D0E773CDFF7,electric_bike,2024-01-05 12:27:58,2024-01-05 12:35:40,1st & O St NW,31519.0,10th & G St NW,31274.0,38.908690,-77.012317,38.898243,-77.026235,casual


# Exploratory Data Analysis  

Let's start creating new variables within our dataset and taking a look at its structure. 

In [5]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436946 entries, 0 to 436945
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             436946 non-null  object 
 1   rideable_type       436946 non-null  object 
 2   started_at          436946 non-null  object 
 3   ended_at            436946 non-null  object 
 4   start_station_name  371894 non-null  object 
 5   start_station_id    371894 non-null  float64
 6   end_station_name    368605 non-null  object 
 7   end_station_id      368500 non-null  float64
 8   start_lat           436946 non-null  float64
 9   start_lng           436946 non-null  float64
 10  end_lat             436558 non-null  float64
 11  end_lng             436558 non-null  float64
 12  member_casual       436946 non-null  object 
dtypes: float64(6), object(7)
memory usage: 43.3+ MB


Start creating variables: 
- Start and End Dates
- Start and End Times
- Convert Station IDs to Character Integers
- Create new geocombined_dfs that have start and end locations so that we can create the neighborhoods that they started and ended in
- 

In [6]:
# Adding time_of_day
def get_time_of_day(hour):
    if 5 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 17:
        return 'afternoon'
    elif 17 <= hour < 21:
        return 'evening'
    else:
        return 'night'

federal_holidays_2024 = [
    '2024-01-01',  # New Year's Day
    '2024-01-15',  # Martin Luther King Jr. Day
    '2024-02-19',  # Presidents' Day
    '2024-05-27',  # Memorial Day
    '2024-06-19',  # Juneteenth National Independence Day
    '2024-07-04',  # Independence Day
    '2024-09-02',  # Labor Day
    '2024-10-14',  # Columbus Day
    '2024-11-11',  # Veterans Day
    '2024-11-28',  # Thanksgiving Day
    '2024-12-25'   # Christmas Day
]

# Adding holiday_weekend
def is_holiday_weekend(date):
    dt = pd.to_datetime(date)
    if dt.strftime('%Y-%m-%d') in federal_holidays_2024 and dt.weekday() in [1, 2, 3]:  # Tuesday, Wednesday, Thursday
        return True
    return False

In [13]:
combined_df["started_at"]= pd.to_datetime(combined_df["started_at"])
combined_df["started_at_date"]= pd.to_datetime(combined_df["started_at"]).dt.date
combined_df["started_at_time"]= pd.to_datetime(combined_df["started_at"]).dt.time
combined_df["ended_at"]= pd.to_datetime(combined_df["ended_at"])
combined_df["ended_at_date"]= pd.to_datetime(combined_df["ended_at"]).dt.date
combined_df["ended_at_time"]= pd.to_datetime(combined_df["ended_at"]).dt.time
combined_df["start_station_id"] = combined_df["start_station_id"].astype(str)
combined_df["end_station_id"] = combined_df["end_station_id"].astype(str)
combined_df["duration"] = pd.to_datetime(combined_df["ended_at"]) - pd.to_datetime(combined_df["started_at"])
combined_df["duration_minutes"] = combined_df.duration.apply(lambda td: td.total_seconds() / 60)
combined_df['time_of_day'] = combined_df['started_at'].apply(lambda x: get_time_of_day(pd.to_datetime(x).hour))
combined_df['day_of_week'] = combined_df['started_at'].apply(lambda x: pd.to_datetime(x).strftime('%A'))
# Adding day_type
combined_df['day_type'] = combined_df['day_of_week'].apply(lambda x: 'weekend' if x in ['Saturday', 'Sunday'] else 'work-day')
# Adding falls_on_holiday
combined_df['falls_on_holiday'] = combined_df['started_at'].apply(lambda x: pd.to_datetime(x).strftime('%Y-%m-%d') in federal_holidays_2024)
combined_df['holiday_weekend'] = combined_df['started_at'].apply(lambda x: is_holiday_weekend(x))
# Filter dataframe to only have rides between 30 seconds and 2 hours
og_df_size = combined_df.shape[0]
combined_df = combined_df.loc[(combined_df['duration_minutes']>=.5) & (combined_df['duration_minutes']<=120) ]

In [14]:
og_df_size
new_df_size = combined_df.shape[0]
new_df_size/og_df_size

1.0

In [15]:
combined_df.describe()

,started_at,ended_at,start_lat,start_lng,end_lat,end_lng,duration,duration_minutes
count,427226,427226,427226.000000,427226.000000,427226.000000,427226.000000,427226,427226.000000
mean,2024-03-17 10:53:56.171096832,2024-03-17 11:08:53.587810816,38.904837,-77.031332,38.903965,-77.031151,0 days 00:14:57.416713870,14.956945
min,2024-03-01 00:00:22,2024-03-01 00:07:47,38.760000,-77.370000,38.710000,-77.440000,0 days 00:00:30,0.500000
25%,2024-03-10 15:43:06.249999872,2024-03-10 15:56:52,38.892223,-77.043074,38.890540,-77.043182,0 days 00:05:49,5.816667
50%,2024-03-17 12:01:29,2024-03-17 12:19:46,38.903750,-77.030000,38.903040,-77.030000,0 days 00:10:07,10.116667
75%,2024-03-24 18:41:54.249999872,2024-03-24 19:00:39.750000128,38.917668,-77.012344,38.915544,-77.012289,0 days 00:17:40,17.666667
max,2024-03-31 23:58:41,2024-04-01 00:50:05,39.125828,-76.820000,39.125828,-76.820000,0 days 02:00:00,120.000000
std,NaN,NaN,0.026159,0.034113,0.026088,0.034024,0 days 00:15:35.989089177,15.599818


Create two new dataframes: 
1. Start times combined_df that has the location they started at if it has a station
2. End times combined_df that has the location they ended at if it has a station 

In [16]:
start_stations = combined_df[["start_station_name","start_station_id","start_lat","start_lng"]]
end_stations = combined_df[["end_station_name","end_station_id","end_lat","end_lng"]]
start_stations = start_stations.loc[start_stations['start_station_name'].notna()]
end_stations = end_stations.loc[end_stations['end_station_name'].notna()]
start_stations.rename(columns = {"start_station_name":"station_name",
                                "start_station_id":"station_id",
                                "start_lat":"lat",
                                 "start_lng":"lng"}, inplace=True)

end_stations.rename(columns = {"end_station_name":"station_name",
                                "end_station_id":"station_id",
                                "end_lat":"lat",
                                 "end_lng":"lng"}, inplace=True )
stations = pd.concat([start_stations, end_stations])
stations = stations.drop_duplicates(subset=["station_name","station_id"])

In [17]:
combined_gdf = gpd.GeoDataFrame(stations, geometry=gpd.points_from_xy(stations.lng, stations.lat), crs="EPSG:4326")

# Pull in Weather Data to add to rides

In [19]:
# Function to fetch weather data
def fetch_weather_data(lat, lon, date, api_key):
    url = "https://api.openweathermap.org/data/3.0/onecall/day_summary?"
    params = {
        'lat': lat,
        'lon': lon,
        'date': date.strftime('%Y-%m-%d'),
        'appid': api_key,
        'units':"imperial"
    }
    response = requests.get(url, params=params)
    return response.json()
# Parameters
api_key = "32964f206a33b141c84d7f13e55a92d4"
dc_lat = 38.8950368
dc_lon = -77.0365427
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 3, 31)

# Create a list of dates for each day within the date range
date_range = [start_date + timedelta(days=x) for x in range((end_date - start_date).days + 1)]


In [20]:
# Fetch data
weather_data = fetch_weather_data(dc_lat, dc_lon, start_date, api_key)

In [ ]:
# Fetch data
#daily_data = []
#for date in date_range:
    weather_data = fetch_weather_data(dc_lat, dc_lon, date, api_key)
    daily_data.append({
        'date': weather_data['date'],
        'min_temp': weather_data['temperature']['min'],
        'max_temp': weather_data['temperature']['max'],
        'afternoon_temp': weather_data['temperature']['afternoon'],
        'night_temp': weather_data['temperature']['night'],
        'evening_temp': weather_data['temperature']['evening'],
        'morning_temp': weather_data['temperature']['morning'],
        'humidity': weather_data['humidity']['afternoon'],
        'pressure': weather_data['pressure']['afternoon'],
        'wind_speed': weather_data['wind']['max']['speed'],
        'wind_direction': weather_data['wind']['max']['direction'],
        'precipitation': weather_data['precipitation']['total'],
        'cloud_cover': weather_data['cloud_cover']['afternoon']
    })


In [21]:
# Process and save data
#weather_df = pd.DataFrame(daily_data)
#weather_df.to_csv("data/raw_weather_data.csv")
weather_df = pd.read_csv("data/raw_weather_data.csv")

In [34]:
weather_df

,Unnamed: 0,date,min_temp,max_temp,afternoon_temp,night_temp,evening_temp,morning_temp,humidity,pressure,wind_speed,wind_direction,precipitation,cloud_cover
0,0,2024-01-01,37.89,41.83,41.68,40.59,38.08,38.53,69.0,1016.0,13.80,320.0,1.92,100.0
1,1,2024-01-02,33.87,44.47,41.04,38.88,39.76,35.73,57.0,1020.0,18.41,330.0,0.00,75.0
2,2,2024-01-03,31.01,47.46,44.62,33.21,41.90,31.42,54.0,1017.0,9.22,180.0,0.00,40.0
3,3,2024-01-04,31.82,42.89,42.89,39.24,37.83,38.59,59.0,1017.0,21.85,330.0,0.00,100.0
4,4,2024-01-05,25.38,41.05,36.84,30.76,34.93,26.01,54.0,1027.0,13.80,330.0,0.00,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,86,2024-03-27,44.87,48.36,46.92,46.89,48.36,45.07,90.0,1018.0,11.50,110.0,13.55,100.0
87,87,2024-03-28,46.31,52.41,49.89,47.79,51.33,47.59,83.0,1016.0,16.11,330.0,5.87,100.0
88,88,2024-03-29,36.84,61.74,55.80,46.74,59.54,36.97,47.0,1010.0,26.46,320.0,0.00,20.0
89,89,2024-03-30,37.51,64.02,61.00,45.63,64.02,37.78,34.0,1012.0,16.11,350.0,0.80,40.0


In [90]:
# Ensure 'started_at' is a datetime
combined_df['started_at'] = pd.to_datetime(combined_df['started_at'])

# Create a 'date' column in datetime format
combined_df['date'] = combined_df['started_at'].dt.date
combined_df['date'] = pd.to_datetime(combined_df['date'])

# Melt the weather data to make times of the day as rows
weather_melted = weather_df.reset_index().melt(id_vars=['date'], 
                                               value_vars=['afternoon_temp', 'night_temp', 'evening_temp', 'morning_temp'], 
                                               var_name='time_of_day', 
                                               value_name='temperature')
# Ensure 'date' is a datetime
weather_melted['date'] = pd.to_datetime(weather_melted['date'])
weather_df['date'] = pd.to_datetime(weather_df['date'])
# Map the time_of_day to match combined_df format
time_mapping = {
    'afternoon_temp': 'afternoon',
    'night_temp': 'night',
    'evening_temp': 'evening',
    'morning_temp': 'morning',
    
}
weather_melted['time_of_day'] = weather_melted['time_of_day'].map(time_mapping)
weather_melted = pd.merge(weather_melted, weather_df[['date','wind_speed','humidity']] , how='left', left_on=['date'], right_on=['date'])

In [91]:
# Merge based on 'date' and 'time_of_day'
merged_df = pd.merge(combined_df, weather_melted, how='left', left_on=['date', 'time_of_day'], right_on=['date', 'time_of_day'])
# Create target variable: 'ended_at_dock' (1 if ended at a dock, 0 if not)
merged_df['ended_at_dock'] = merged_df['end_station_name'].apply(lambda x: 1 if pd.notnull(x) else 0)

# Start Analysis for Underserved Rides 

The idea for this analysis is that if a rider completes their ride outside of a dock, they either had to find a place nearby to park their bike because a nearby dock was full OR because there simply was not a dock nearby. According to the National Association of City Transportation Officials's [Bike Share Station Siting Guide](https://nacto.org/publication/bike-share-station-siting-guide/), docks should be stationed approximately 1,000 feet, or a 5-minute walk from one another. 

In [92]:
# Create a neighborhood variable that tells us the neighborhood with which a bikeride started. We'll use this as a fixed effect variable 
neighborhoods = gpd.read_file("data/DC_Health_Planning_Neighborhoods.geojson")
merged_gdf =  gpd.GeoDataFrame(merged_df, geometry=gpd.points_from_xy(merged_df.start_lng, merged_df.start_lat), crs="EPSG:4326")
merged_gdf = merged_gdf.sjoin(neighborhoods, how="left")


In [99]:
# Remove all rides that did not start in a DC neighborhood 
merged_gdf = merged_gdf.loc[merged_gdf['OBJECTID'].notnull()]
# only use the columns that I think that we want to use for the design matrix 
mgdf_start_dm = merged_gdf[['rideable_type', 'started_at', 'start_station_name', 'start_station_id', 'member_casual', 'time_of_day', 'day_of_week','day_type', 'falls_on_holiday', 'holiday_weekend', 
                            'temperature', 'wind_speed','humidity','DC_HPN_NAME','end_station_name', 'ended_at_dock']]
# Ensure 'date' is a datetime
mgdf_start_dm['started_at'] = pd.to_datetime(mgdf_start_dm['started_at'])
mgdf_start_dm['started_at_hour'] = mgdf_start_dm['started_at'].dt.hour
# Create target variable: 'ended_at_dock' (1 if ended at a dock, 0 if not)
mgdf_start_dm['ended_at_dock'] = mgdf_start_dm['end_station_name'].apply(lambda x: 1 if pd.notnull(x) else 0)

/tmp/ipykernel_3921/1233916429.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mgdf_start_dm['started_at'] = pd.to_datetime(mgdf_start_dm['started_at'])
/tmp/ipykernel_3921/1233916429.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mgdf_start_dm['started_at_hour'] = mgdf_start_dm['started_at'].dt.hour
/tmp/ipykernel_3921/1233916429.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [100]:
from sklearn.model_selection import train_test_split

In [101]:
# Split the data into training and testing sets
X = mgdf_start_dm.drop(['started_at','end_station_name','ended_at_dock'], axis=1)
y = mgdf_start_dm['ended_at_dock']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature Engineering 

In [102]:
X.head()

,rideable_type,start_station_name,start_station_id,member_casual,time_of_day,day_of_week,day_type,falls_on_holiday,holiday_weekend,temperature,wind_speed,humidity,DC_HPN_NAME,started_at_hour
0,classic_bike,22nd & H St NW,31127.0,member,evening,Monday,work-day,False,False,55.42,31.07,36.0,GWU,17
3,classic_bike,Columbia Rd & Belmont St NW,31113.0,member,evening,Friday,work-day,False,False,59.54,26.46,47.0,ADAMS MORGAN,18
4,classic_bike,Columbia Rd & Belmont St NW,31113.0,member,morning,Sunday,weekend,False,False,31.26,20.71,41.0,ADAMS MORGAN,11
6,classic_bike,22nd & H St NW,31127.0,member,afternoon,Friday,work-day,False,False,45.19,11.50,44.0,GWU,13
8,classic_bike,15th & M St NW,31298.0,member,afternoon,Wednesday,work-day,False,False,53.76,12.66,94.0,CHINATOWN,16


In [107]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Define columns to be used in the model
numerical_features = [ 'temperature', 'humidity', 'wind_speed']
categorical_features = ['rideable_type','started_at_hour','start_station_id', 'time_of_day', 'member_casual', 
                        'day_of_week', 'day_type', 'falls_on_holiday', 'holiday_weekend','DC_HPN_NAME']

# Preprocessing for numerical data
numerical_transformer = StandardScaler()

# Preprocessing for categorical data
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Define the model pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

In [108]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Add classifier to the pipeline
pipeline.steps.append(('classifier', RandomForestClassifier(random_state=42)))

In [109]:
# Train the model
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['temperature', 'humidity',
                                                   'wind_speed']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['rideable_type',
                                                   'started_at_hour',
                                                   'start_station_id',
                                                   'time_of_day',
                                                   'member_casual',
                                                   'day_of_week', 'day_type',
                                                   'falls_on_holiday',
                                                   'holiday_weekend',
                                                   'DC_HPN_NAME'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [110]:
# Evaluate the model
y_pred = pipeline.predict(X_test)


              precision    recall  f1-score   support

           0       0.54      0.41      0.46     11534
           1       0.90      0.94      0.92     64481

    accuracy                           0.86     76015
   macro avg       0.72      0.67      0.69     76015
weighted avg       0.84      0.86      0.85     76015



In [112]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.54      0.41      0.46     11534
           1       0.90      0.94      0.92     64481

    accuracy                           0.86     76015
   macro avg       0.72      0.67      0.69     76015
weighted avg       0.84      0.86      0.85     76015



## Analysis

    - The model has high precision (0.90) and recall (0.94) for class 1 (ending at a dock), indicating it performs well in predicting rides that end at a dock.
    - The model struggles with class 0 (not ending at a dock), with lower precision (0.54) and recall (0.41). This suggests the model often misclassifies rides that end outside a dock or misses them altogether.
    - The imbalance in support (11,534 vs. 64,481) indicates class imbalance, which could be affecting model performance.

In [115]:
X_train

,rideable_type,start_station_name,start_station_id,member_casual,time_of_day,day_of_week,day_type,falls_on_holiday,holiday_weekend,temperature,wind_speed,humidity,DC_HPN_NAME,started_at_hour
364851,electric_bike,NaN,nan,member,morning,Thursday,work-day,False,False,32.65,18.41,34.0,16th ST HEIGHTS,7
343769,classic_bike,Maine Ave & 9th St SW,31646.0,member,evening,Thursday,work-day,False,False,59.56,18.41,65.0,SW/WATERFRONT,19
360503,electric_bike,10th & Quincy St NE / Turkey Thicket Rec,31541.0,member,evening,Wednesday,work-day,False,False,70.18,11.50,31.0,MICHIGAN PARK,17
257086,classic_bike,14th & Otis Pl NW,31131.0,member,evening,Thursday,work-day,False,False,76.73,11.50,32.0,COLUMBIA HEIGHTS,17
192119,classic_bike,4th & G St SW,31666.0,member,morning,Tuesday,work-day,False,False,48.09,13.80,88.0,SW/WATERFRONT,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290018,electric_bike,1st & M St NE,31603.0,member,evening,Wednesday,work-day,False,False,61.00,25.32,33.0,UNION STATION,18
411494,classic_bike,4th & Florida Ave NE,31532.0,casual,afternoon,Tuesday,work-day,False,False,51.08,13.80,88.0,TRINIDAD,12
152633,classic_bike,14th & V St NW,31101.0,member,morning,Tuesday,work-day,False,False,35.22,17.27,42.0,U ST/PLEASANT,7
168579,electric_bike,NaN,nan,casual,afternoon,Thursday,work-day,False,False,49.89,16.11,83.0,SW/WATERFRONT,16


In [114]:
from imblearn.over_sampling import SMOTE

# Oversample the minority class
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Train the model on the resampled data
pipeline.set_params(classifier=RandomForestClassifier(random_state=42))
pipeline.fit(X_resampled, y_resampled)

# Evaluate the model
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

ValueError: could not convert string to float: 'electric_bike'